<p align='center'><img src='https://i.hizliresim.com/8a6tv6h.gif'></p>

**GColab**
==
**GColab**, Google Colab'da bulunan Nvidia T4 GPU'nun gücünü kullanarak otomatik olarak görüntü kalitesini artırır. Nvidia T4 GPU'sunu kullanarak **Remini-AI**, görüntü kalitesini Google Colab ortamında üstün ve daha hızlı sonuçlar elde etmek üzere geliştirilmiştir.

In [ ]:
#@title **Paket Kurulumu**

!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

from IPython.display import clear_output
clear_output()
print("Paket kurulumu başarılı")

In [ ]:
#@title **RealESRGAN İle Remini-AI Modeli Oluşturun**
from RealESRGAN import RealESRGAN
from IPython.display import clear_output
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4" #@param ["2", "4", "8"] {allow-input: false}

try:
    model = RealESRGAN(device, scale=int(model_scale))
    model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')
    clear_output()
    print(f"Model Ölçeği {model_scale} Başarıyla Yüklendi!")
except Exception as e:
    clear_output()
    print('Bir şeyler yanlış gitti..')
    print(e)

In [ ]:
#@title **Yükle ve Sonucu Al**
import os
import requests
from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile
from PIL import Image
from IPython.display import clear_output
import numpy as np

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join('results/', os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, işleniyor {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'{member.name} dosyası açılamıyor, atlanıyor')
            continue

        img_sr = model.predict(np.array(img_lr))
        # save_tar'a ekleme
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()
    print(f'Tamamlandı! Arşiv, {result_tar_path} konumuna kaydedildi')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join('results/', os.path.basename(filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        print(f'Tamamlandı! Görüntü, {result_image_path} konumuna kaydedildi')
        print('Fotoğrafı telegra.ph yükle...')

        # İşlenmiş görüntüyü belirtilen URL'ye yükle
        url = 'https://telegra.ph/upload'
        with open(result_image_path, 'rb') as file:
            files = {'file': file}
            response = requests.post(url, files=files)
            response_json = response.json()
            if response.status_code == 200:
                clear_output()
                print('Sonuçları Görüntüle')
                print('')
                print('Dosya Adı:', response_json['result']['originalname'])
                print('Dosya Boyutu:', response_json['result']['filesize'])
                print('Dosya URLsi:', response_json['result']['url_file'])
            else:
                clear_output()
                print('Yükleme başarısız. Yanıt:', response.text)

        # Orijinal ve işlenmiş dosyaları sil
        os.remove(filename)
        os.remove(result_image_path)

# Dosyaları yükle
try:
    uploaded = files.upload()
    for filename in uploaded.keys():
        print('İşleniyor:', filename)
        process_input(filename)
except KeyboardInterrupt:
    print('Yükleme iptal edildi.')

# upload_folder ve result_folder dizinlerini kaldır
shutil.rmtree(upload_folder)
shutil.rmtree(result_folder)

**Katkıda Bulunanlar✨**
==
**@codermert**
